In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
train_dataset = datasets.MNIST(root='/data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='/data', train=True, transform=transforms.ToTensor(), download=True)

In [24]:
batch_size = 100
n_iters = 3000
n_epochs = n_iters / (len(train_dataset) / batch_size)
n_epochs = int(n_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

print(len(train_dataset))

60000


In [25]:
class LSTM_model(nn.Module):
    def __init__(self, input_dims, hidden_dims, n_layers, output_dims):
        super(LSTM_model, self).__init__()
        self.hidden_dims = hidden_dims
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dims, hidden_dims, n_layers, batch_first=True) # in/output tensors of form (batch_dim, seq_d, feature_d)
        self.fc = nn.Linear(hidden_dims, output_dims)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dims).requires_grad_().to(device) #hidden state
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dims).requires_grad_().to(device) #cell state
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) #100,28,100 -> 100,100: only want prev. hidden state
        return out

In [26]:
input_dims = 28
hidden_dims = 100
n_layers = 1
output_dims = 10

model = LSTM_model(input_dims, hidden_dims, n_layers, output_dims)
model.to(device)

LSTM_model(
  (lstm): LSTM(28, 100, batch_first=True)
  (fc): Linear(in_features=100, out_features=10, bias=True)
)

In [29]:
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.1)

In [31]:
n_epochs = 10
seq_dims = 28
iter = 0

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, seq_dims, input_dims).requires_grad_().to(device)
        labels = labels.to(device)
        optimiser.zero_grad()
        outputs = model(images) #fp
        loss = criterion(outputs, labels)
        loss.backward() #grads
        optimiser.step() #update params

        iter += 1


        if iter % 500 == 0:
            correct = 0
            total = 0

            for images, labels in test_loader:
                images = images.view(-1, seq_dims, input_dims).to(device)
                labels = labels.to(device)
                outputs = model(images) #fp for outputs
                _, pred = torch.max(outputs.data, 1) #preds from max val
                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


NameError: name 'predicted' is not defined